<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=834a713af93180fd065475e35cad7c74c347b11b2605ea59bce35e0044491c3e
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-20 14:53:23
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.37 C
-------------------
Today PnL: -2.52 L (-1.81%)
Current PnL: -32.50 L (-20.73%)
CY Booked + Current PnL: -18.21 L (-11.61%)
-------------------
Total profit:  1.35 L
Total loss:  -33.85 L
-------------------
Total Booked + Current PnL: 8.77 L (6.75%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.46%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 97.85 L (71.65%)
Deployed:  1.30 C
Current:  1.37 C
CAGR/XIRR %: 2.7%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-1.00,14.82,5.73,21.40,17463.0,39336.0,304762.0,6.58,56.0,X-LC,6.91,14.0,2.25,2.23,22.08,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,1.49,-23.46,30.76,0.09,23728.0,-23638.0,77139.0,79.08,36.0,M-SC,4.73,253.0,-1.00,0.56,1.49,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-2.71,2.91,15.85,19.23,24543.0,4384.0,154845.0,21.35,55.0,L-LC,10.39,182.0,0.18,1.13,24.81,XY25,ATH,MINING
49,MASFIN,398.61,-1.53,-6.37,30.35,22.05,27843.0,-6240.0,91740.0,-19.15,41.0,H-SC,4.98,168.0,-0.22,0.67,33.35,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.75,-18.97,23.43,0.02,33141.0,-33112.0,141447.0,115.96,41.0,M-SC,9.09,240.0,-1.00,1.04,23.03,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJOOENG,143.10,4.39,-30.35,100.45,39.61,71711.0,-31110.0,71390.0,-52.96,57.0,H-SC,28.92,136.0,-0.43,0.52,15.56,AR,ATH,MISC
33,HINDUNILVR,2922.00,2.27,-6.10,21.05,13.67,50813.0,-15673.0,241390.0,-13.11,60.0,X-LC,2.56,9.0,-0.31,1.77,13.96,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,1.49,-23.46,30.76,0.09,23728.0,-23638.0,77139.0,79.08,36.0,M-SC,4.73,253.0,-1.00,0.56,1.49,OX40N,NTT,DURABLES
67,SONACOMS,806.63,0.17,-22.43,79.89,39.54,62690.0,-22689.0,78470.0,-39.30,31.0,M-SC,3.16,227.0,-0.36,0.57,10.65,AR,ATH,AUTO
28,GREENPANEL,537.00,0.14,-43.76,154.50,43.14,159086.0,-80110.0,102968.0,130.62,28.0,M-SC,14.71,231.0,-0.50,0.75,0.14,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HAVELLS,2069.16,-6.43,-13.31,52.82,32.48,150188.0,-43651.0,284340.0,-17.74,33.0,X-MC,5.13,69.0,-0.29,2.08,0.00,X40,ATH,ELECTRICAL
13,BSOFT,831.70,-5.42,-26.98,106.51,50.79,109387.0,-37952.0,102701.0,-6.46,41.0,H-SC,8.73,171.0,-0.35,0.75,19.58,XR,ATH,IT
18,COFFEEDAY,80.00,-4.69,-44.62,144.27,35.27,90717.0,-50669.0,62880.0,-57.75,39.0,L-SC,5.47,268.0,-0.56,0.46,53.90,XR,NTT,HOTELS
82,VOLTAS,1856.00,-4.53,4.45,39.02,45.20,78143.0,8523.0,200265.0,-5.78,35.0,X-MC,5.71,80.0,0.11,1.47,11.54,XY25,NTT,AC
40,INDUSINDBK,1800.00,-4.40,-32.96,98.23,32.89,50843.0,-25448.0,51759.0,-738.98,55.0,L-MC,10.22,259.0,-0.50,0.38,42.54,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-3.00,-7.10,123.81,107.91,169812.0,-10489.0,137155.0,-24.60,38.0,M-SC,9.93,220.0,-0.06,1.00,2.27,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.75,-18.97,23.43,0.02,33141.0,-33112.0,141447.0,115.96,41.0,M-SC,9.09,240.0,-1.00,1.04,23.03,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-2.47,-21.81,68.85,32.03,70973.0,-28751.0,103084.0,-15.37,49.0,H-SC,6.37,157.0,-0.41,0.75,15.46,OX40N,NTT,IT
77,TTKPRESTIG,770.00,1.49,-23.46,30.76,0.09,23728.0,-23638.0,77139.0,79.08,36.0,M-SC,4.73,253.0,-1.00,0.56,1.49,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.46,-24.24,49.77,13.47,101685.0,-65358.0,204309.0,-68.90,41.0,H-SC,4.23,159.0,-0.64,1.50,3.89,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,-2.71,2.91,15.85,19.23,24543.0,4384.0,154845.0,21.35,55.0,L-LC,10.39,182.0,0.18,1.13,24.81,XY25,ATH,MINING


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-2.20,-1.34,74.60,72.27,124718.0,-2266.0,167182.0,-15.15,32.0,M-LC,5.11,101.0,-0.02,1.22,4.76,XR,NTT,IT
25,FINCABLES,1641.55,-3.00,-7.10,123.81,107.91,169812.0,-10489.0,137155.0,-24.60,38.0,M-SC,9.93,220.0,-0.06,1.00,2.27,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-2.04,-7.63,123.79,106.72,141033.0,-9407.0,113929.0,-54.25,44.0,H-SC,8.62,138.0,-0.07,0.83,13.97,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.44,-7.49,46.61,35.63,90364.0,-15702.0,193872.0,-20.73,35.0,H-MC,4.48,119.0,-0.17,1.42,7.93,AR,ATH,PHARMA
37,IEX,219.00,-3.36,-10.15,65.41,48.63,117520.0,-20287.0,179667.0,-40.27,40.0,H-SC,13.17,137.0,-0.17,1.32,0.30,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-1.05,-32.00,131.17,57.19,290881.0,-104368.0,221759.0,-67.84,15.0,X-MC,12.35,58.0,-0.36,1.62,0.0,XY24,NTT,FMCG
43,ITC,452.00,-1.64,-19.67,37.91,10.78,73183.0,-47273.0,193045.0,-52.19,17.0,X-LC,11.77,1.0,-0.65,1.41,0.0,X40,NTT,FMCG
58,RELAXO,1176.00,-2.58,-52.50,220.61,52.31,152128.0,-76202.0,68958.0,-49.38,19.0,X-SC,8.88,92.0,-0.50,0.50,0.0,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.75,-42.72,136.04,35.20,100267.0,-54966.0,73704.0,-3.39,21.0,X-SC,17.90,93.0,-0.55,0.54,0.0,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-1.37,-10.12,49.29,34.19,94597.0,-21600.0,191920.0,-36.80,22.0,X-MC,5.12,60.0,-0.23,1.41,0.0,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.59,-1.70,-14.49,38.65,18.55,125001.0,-54818.0,323419.0,-26.21,44.0,X-LC,1.22,3.0,-0.44,2.37,10.03,X40,ATH,IT
20,DABUR,735.00,-0.95,-0.70,44.71,43.70,110135.0,-1736.0,246332.0,-8.87,47.0,X-MC,2.08,73.0,-0.02,1.80,14.25,XY24,BTT,FMCG
45,JIOFIN,387.00,-2.63,-13.67,44.19,24.48,102001.0,-36541.0,230824.0,-15.14,27.0,X-LC,2.12,37.0,-0.36,1.69,33.79,XY24,BTT,FINANCE
78,UNITDSPR,1644.00,-0.15,-2.83,24.28,20.77,55242.0,-6622.0,227522.0,-11.23,34.0,X-MC,2.22,70.0,-0.12,1.67,3.89,X40N,NTT,BREWERIES
35,ICICIGI,2252.93,-0.85,-1.01,20.55,19.34,40708.0,-2016.0,198093.0,-21.92,36.0,X-MC,2.37,71.0,-0.05,1.45,14.77,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-1.05,-32.00,131.17,57.19,290881.0,-104368.0,221759.0,-67.84,15.0,X-MC,12.35,58.0,-0.36,1.62,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.75,-42.72,136.04,35.20,100267.0,-54966.0,73704.0,-3.39,21.0,X-SC,17.90,93.0,-0.55,0.54,0.0,X40,NTT,FOOTWEAR
9,BAJAJHLDNG,14451.52,-0.58,-5.35,36.52,29.22,65722.0,-10163.0,179962.0,-8.00,33.0,X-LC,12.22,29.0,-0.15,1.32,0.0,X40,ATH,FINANCE
8,BAJAJHFL,181.50,-1.39,-22.84,101.49,55.47,190524.0,-55559.0,187727.0,-31.69,24.0,X-MC,16.46,63.0,-0.29,1.37,0.0,X40N,ATH,FINANCE
31,HAVELLS,2069.16,-6.43,-13.31,52.82,32.48,150188.0,-43651.0,284340.0,-17.74,33.0,X-MC,5.13,69.0,-0.29,2.08,0.0,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-2.45,-32.56,111.80,42.84,49015.0,-21164.0,43842.0,-55.92,35.0,X-SC,23.36,83.0,-0.43,0.32,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-2.58,-52.50,220.61,52.31,152128.0,-76202.0,68958.0,-49.38,19.0,X-SC,8.88,92.0,-0.50,0.50,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.75,-42.72,136.04,35.20,100267.0,-54966.0,73704.0,-3.39,21.0,X-SC,17.90,93.0,-0.55,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-3.98,-10.66,38.88,24.08,44098.0,-13530.0,113420.0,-17.84,30.0,X-SC,6.46,91.0,-0.31,0.83,6.87,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-1.07,-18.07,74.93,43.32,99987.0,-29432.0,133440.0,-29.54,44.0,X-SC,8.17,90.0,-0.29,0.98,2.46,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-1.64,-19.67,37.91,10.78,73183.0,-47273.0,193045.0,-52.19,17.0,X-LC,11.77,1.0,-0.65,1.41,0.00,X40,NTT,FMCG
75,TMPV,600.00,-1.60,-30.62,77.25,22.98,121071.0,-69172.0,156726.0,-27.96,31.0,X-LC,3.61,2.0,-0.57,1.15,0.00,XY24,NTT,AUTO
73,TCS,4311.59,-1.70,-14.49,38.65,18.55,125001.0,-54818.0,323419.0,-26.21,44.0,X-LC,1.22,3.0,-0.44,2.37,10.03,X40,ATH,IT
81,VBL,671.64,-0.58,-0.39,36.21,35.67,113916.0,-1244.0,314598.0,-12.17,53.0,X-LC,2.98,5.0,-0.01,2.30,13.41,X40N,ATH,FMCG
41,INFY,1972.00,-1.05,13.24,18.55,34.24,65727.0,41438.0,354326.0,-11.70,59.0,X-LC,7.40,6.0,0.63,2.59,22.62,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-4.40,-32.96,98.23,32.89,50843.0,-25448.0,51759.0,-738.98,55.0,L-MC,10.22,259.0,-0.50,0.38,42.54,XR,NTT,BANKS
17,COALINDIA,484.83,-2.71,2.91,15.85,19.23,24543.0,4384.0,154845.0,21.35,55.0,L-LC,10.39,182.0,0.18,1.13,24.81,XY25,ATH,MINING
36,ICICIPRULI,790.00,-0.63,8.69,20.28,30.73,43423.0,17112.0,214117.0,-16.68,47.0,X-MC,7.41,79.0,0.39,1.57,22.56,X40,ATH,INSURANCE
5,ASIANPAINT,3460.25,-2.29,-5.94,28.59,20.95,67703.0,-14960.0,236808.0,-6.96,38.0,X-LC,12.75,36.0,-0.22,1.73,27.58,X40,ATH,PAINTS
32,HCLTECH,1908.19,-1.40,9.86,12.72,23.83,33806.0,23846.0,265770.0,14.88,62.0,X-LC,6.37,13.0,0.71,1.95,27.51,X40,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-4.16,-30.85,63.27,12.90,95595.0,-67405.0,151091.0,122.36,53.0,H-SC,7.77,173.0,-0.71,1.11,25.63,OX40N,NTT,FINANCE
56,RAJOOENG,143.10,4.39,-30.35,100.45,39.61,71711.0,-31110.0,71390.0,-52.96,57.0,H-SC,28.92,136.0,-0.43,0.52,15.56,AR,ATH,MISC
17,COALINDIA,484.83,-2.71,2.91,15.85,19.23,24543.0,4384.0,154845.0,21.35,55.0,L-LC,10.39,182.0,0.18,1.13,24.81,XY25,ATH,MINING
62,SATIN,274.00,-0.98,-16.88,82.67,51.83,189852.0,-46634.0,229650.0,-25.10,59.0,H-SC,3.56,148.0,-0.25,1.68,11.87,XY24,NTT,FINANCE
40,INDUSINDBK,1800.00,-4.40,-32.96,98.23,32.89,50843.0,-25448.0,51759.0,-738.98,55.0,L-MC,10.22,259.0,-0.50,0.38,42.54,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.61
1,25,44.85
2,50,76.74


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.43
MC    30.67
LC    26.85
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.20
X40      24.15
X40N     12.91
XY25     10.94
XR        8.75
AR        8.42
OX40N     7.63
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.29
X-LC    23.45
H-SC    22.73
M-SC    11.05
X-SC     7.87
H-MC     4.54
M-MC     1.46
M-LC     1.22
L-LC     1.13
H-LC     1.05
L-SC     0.78
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.45,-10.45,46.69
IT,12.83,-21.92,82.82
FINANCE,11.08,-17.75,67.17
MISC,7.05,-34.55,89.59
PAINTS,5.81,-16.55,33.86
ELECTRICAL,5.76,-19.24,62.92
INSURANCE,4.76,-4.43,40.16
PHARMA,3.96,-8.79,43.04
BANKS,2.78,-40.91,126.29


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3391824.0,21
AR,1419843.0,10
XR,1296113.0,12
X40,1212958.0,15
X40N,977365.0,9
OX40N,749288.0,10
XY25,447094.0,7
SR,291004.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3688896.0,24
X-MC,1716583.0,16
M-SC,1548974.0,15
X-LC,1009874.0,13
X-SC,843431.0,8
H-MC,457872.0,3
L-SC,184392.0,2
M-LC,124718.0,1
H-LC,89980.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1292053.0      6
           AR         947881.0      5
M-SC       XY24       884952.0      6
H-SC       XR         836202.0      7
X-MC       X40        581145.0      7
           XY24       466733.0      3
X-LC       X40        431142.0      6
X-MC       X40N       431075.0      4
M-SC       OX40N      333857.0      5
X-SC       X40N       330168.0      3
H-SC       OX40N      321756.0      4
X-SC       XY24       312592.0      3
H-SC       SR         291004.0      2
H-MC       AR         245450.0      2
X-MC       XY25       237630.0      2
X-LC       XY24       223072.0      2
           X40N       216122.0      2
H-MC       XY24       212422.0      1
X-SC       X40        200671.0      2
M-SC       XR         193633.0      2
X-LC       XY25       139538.0      3
M-SC       AR         136532.0      2
M-LC       XR         124718.0      1
L-SC       OX40N       93675.0      1
           XR          90717.0      1
H-LC       AR          89980.0      1
L-MC       XR          50843.0      1
M-MC       XY25        45383.0      1
L-LC       XY25        24543.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
